In [1]:
import pandas as pd
from pvtools import Sequence
import json

In [2]:
with open('data.json') as f:
    data = json.load(f)

In [3]:
ng = Sequence(data['FASTA']['NG_011745.1'])
nm = Sequence(data['FASTA']['NM_006446.5'])

In [ ]:
# for pos, ref in ng_dict.items():
#     print(ng.data[pos-1] == ref)

In [ ]:
# with open('data.json', 'w') as f:
#     json.dump(data, f, indent=4)

# Test 1.

In [ ]:
df = pd.read_table('manual.tsv')
df = df[df['NG_011745.1 (start=1)'] != '.']
df['Ref'] = df['SNP'].str.split('>').str[0]
df['NG_011745.1 (start=1)'] = df['NG_011745.1 (start=1)'].astype(int)
ng_dict = dict(zip(df['NG_011745.1 (start=1)'], df['Ref']))
results = []
for pos, allele in ng_dict.items():
    results.append(ng.seq[pos-1] == allele)
if all(results):
    print('All alleles passed')

# Test 2.

In [ ]:
ng.transcribe() == nm.seq

# Test 3.

In [ ]:
df = pd.read_table('manual.tsv')

b = []

for item in df['Impact'].to_list():
    if any(x.isdigit() for x in item):
        b.append(item)
        
pos_list = []

for item in b:
    pos_list.append(int(''.join([x for x in item if x.isdigit()])))

ref_aa = []

for pos in pos_list:
    ref_aa.append(data['FASTA']['NG_011745.1']['Protein'][pos-1])

for i in range(len(b)):
    item = b[i]
    pos = pos_list[i]
    aa = item.split(str(pos))[0]
    print(aa == ref_aa[i], aa, ref_aa[i], pos)

In [ ]:
4195 - 5001 - 105 + 1

In [ ]:
5001 - 833 + 105 -1

In [ ]:
5001 - 910 + 105 -1

In [ ]:
105, 2180

In [7]:
ng.liftover(48402, nm.data['CDSStarts'][0])


245

In [ ]:
ng.data['CDSEnds'][0] - ng.data['CDSStarts'][0]